<a href="https://colab.research.google.com/github/panchambanerjee/finetuning_expts/blob/main/fdp_demo_2025_07_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from huggingface_hub import notebook_login

notebook_login()

# 1. HuggingFace

In [43]:
# ! pip install transformers datasets evaluate

In [44]:
!pip install -U transformers datasets evaluate huggingface_hub fsspec bitsandbytes

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [45]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [46]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [47]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [48]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [49]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [51]:
import numpy as np
from evaluate import load

metric = load("accuracy")

In [52]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [53]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [54]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [55]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.152035,0.508000
2,No log,1.011878,0.553000
3,No log,0.984914,0.599000


TrainOutput(global_step=375, training_loss=1.0663355305989584, metrics={'train_runtime': 173.4289, 'train_samples_per_second': 17.298, 'train_steps_per_second': 2.162, 'total_flos': 789354427392000.0, 'train_loss': 1.0663355305989584, 'epoch': 3.0})

### Using `trainer.predict()`

In [56]:
# Example of using trainer.predict() on the evaluation dataset
predictions = trainer.predict(small_eval_dataset)

# The predictions object contains logits and labels
print("Predictions (logits):", predictions.predictions)
print("True labels:", predictions.label_ids)

# You can apply argmax to get the predicted class
predicted_classes = np.argmax(predictions.predictions, axis=-1)
print("Predicted classes:", predicted_classes)

Predictions (logits): [[-2.1069226  -2.0572217  -0.7139882   2.1593406   3.3181932 ]
 [-2.9521267  -1.2116668   1.2207041   2.8832457   0.39842263]
 [ 3.204063    0.7791082  -0.6969576  -1.477277   -1.24451   ]
 ...
 [-3.0011094  -1.2470292   1.4384596   2.7416234   0.27718008]
 [-2.0034444  -0.0376083   2.3053565   0.98711145 -1.1610242 ]
 [ 1.0362257   2.6441824   1.0748065  -1.4263301  -2.4597466 ]]
True labels: [2 4 1 4 3 4 2 3 2 3 0 0 3 2 2 1 3 1 2 2 1 2 3 1 1 3 4 0 0 2 2 2 1 3 4 0 0
 1 3 2 0 2 0 0 3 0 3 2 3 0 1 1 3 3 4 4 1 4 1 3 1 0 0 1 4 1 4 3 2 4 1 0 3 3
 4 1 2 1 0 4 4 4 2 3 3 1 4 0 4 2 3 0 0 0 3 4 0 0 1 4 4 0 0 1 1 0 4 2 2 1 1
 4 0 4 0 3 2 0 4 4 4 2 0 0 0 1 3 0 2 0 3 2 2 2 0 3 4 3 0 1 0 1 0 0 4 3 3 1
 3 0 3 4 0 2 1 3 1 3 1 3 1 0 2 0 1 1 0 0 3 3 3 3 3 1 1 0 0 2 3 3 3 1 1 3 4
 0 1 1 2 2 1 3 0 2 1 1 4 0 2 4 1 1 2 3 3 2 0 1 2 4 4 1 4 2 3 0 1 0 0 2 4 3
 3 0 1 1 2 0 4 0 3 0 3 2 3 1 2 4 4 2 1 0 3 1 1 1 3 0 3 0 0 2 0 3 0 3 2 4 2
 2 4 4 1 1 4 4 4 0 4 0 3 1 0 1 2 2 3 0 1 0 4 1 0 4 3 0 0

### Saving and Loading the Model

In [63]:
# Save the model and tokenizer properly
model_save_path = "my_fine_tuned_model_complete"

# Save the model
trainer.save_model(model_save_path)

# Save the tokenizer as well
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to my_fine_tuned_model_complete


### Using a Hugging Face Pipeline for Inference

In [40]:
# import os
# print("Current directory:", os.getcwd())
# print("Files in current directory:", os.listdir('.'))
# print("Model directory exists:", os.path.exists('my_fine_tuned_model'))

Current directory: /content
Files in current directory: ['.config', 'wandb', 'my_fine_tuned_model', 'test_trainer', 'sample_data']
Model directory exists: True


In [65]:
from transformers import pipeline

# Load your fine-tuned model
classifier = pipeline("sentiment-analysis", model="my_fine_tuned_model_complete")

# Example 1: Very positive review
text1 = "This product is absolutely amazing! Best purchase I've ever made."
result1 = classifier(text1)
print(f"Text: {text1}")
print(f"Classification result: {result1}")
print("-" * 60)

# Example 2: Very negative review
text2 = "Terrible quality, waste of money. I hate this so much!"
result2 = classifier(text2)
print(f"Text: {text2}")
print(f"Classification result: {result2}")
print("-" * 60)

# Example 3: Neutral/mixed sentiment
text3 = "The product is okay, nothing special but does the job."
result3 = classifier(text3)
print(f"Text: {text3}")
print(f"Classification result: {result3}")
print("-" * 60)

# Example 4: Positive but mild
text4 = "Pretty good service, I'm satisfied with my experience."
result4 = classifier(text4)
print(f"Text: {text4}")
print(f"Classification result: {result4}")
print("-" * 60)

# Example 5: Negative but not extreme
text5 = "Not impressed with this. Expected better for the price."
result5 = classifier(text5)
print(f"Text: {text5}")
print(f"Classification result: {result5}")
print("-" * 60)

# Bonus: Process multiple texts at once
texts_batch = [
    "Love this! Highly recommend to everyone.",
    "Could be better, but it's acceptable.",
    "Worst experience ever, avoid at all costs!"
]

results_batch = classifier(texts_batch)
print("Batch processing results:")
for text, result in zip(texts_batch, results_batch):
    print(f"Text: {text}")
    print(f"Result: {result}")
    print("-" * 40)

Device set to use cuda:0


Text: This product is absolutely amazing! Best purchase I've ever made.
Classification result: [{'label': 'LABEL_4', 'score': 0.9096513986587524}]
------------------------------------------------------------
Text: Terrible quality, waste of money. I hate this so much!
Classification result: [{'label': 'LABEL_0', 'score': 0.9363799095153809}]
------------------------------------------------------------
Text: The product is okay, nothing special but does the job.
Classification result: [{'label': 'LABEL_1', 'score': 0.7945865988731384}]
------------------------------------------------------------
Text: Pretty good service, I'm satisfied with my experience.
Classification result: [{'label': 'LABEL_3', 'score': 0.49570536613464355}]
------------------------------------------------------------
Text: Not impressed with this. Expected better for the price.
Classification result: [{'label': 'LABEL_0', 'score': 0.6391899585723877}]
------------------------------------------------------------
Ba

# 2. Unsloth

In [66]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [68]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [4]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/51760 [00:00<?, ? examples/s]

In [5]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.577000
2,2.105400
3,1.676900
4,1.918600
5,1.752100
6,1.513300
7,1.140000
8,1.337300
9,1.197200
10,1.137400


In [6]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

LlamaForCausalLM has no `_prepare_4d_causal_attention_mask_with_cache_position` method defined in its base modeling class. Compiled forward passes will be sub-optimal. If you're writing code, see Llama for an example implementation. If you're a user, please report this issue on GitHub.


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025']

In [7]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the famous expression.", # instruction
        "The quick brown fox ", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the famous expression.\n\n### Input:\nThe quick brown fox \n\n### Response:\nThe quick brown fox jumped over the lazy dog.<|end_of_text|>']

In [7]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')